In [1]:
import random
import numpy as np

prime = 47  # The order of our Finit Field 

# Our Polynomial is 3 * x^0 + 7 * x^1 + 11 * x^2 + 5 * x^3 ....
coefficients = [3, 7, 11, 5, 2, 8, 4, 9, 1]
message_length = 3 #  len(coefficients)**0.5
code_word_length = message_length * 2
test_message = coefficients[:message_length]

# The number of columns need to be checked
sec_num = 2

# The point we want to evaluate
open_point = 5

# Utils

In [2]:
def generate_random_polynomial(p, deg):
    n = deg + 1
    if int(n**0.5)**2 != n:
        raise ValueError(f"{n} is not a perfect square")
    return [random.randint(0, p - 1) for _ in range(n)]

def add_polynomials(poly1, poly2, p):
    max_len = max(len(poly1), len(poly2))
    poly1 += [0] * (max_len - len(poly1))
    poly2 += [0] * (max_len - len(poly2))
    return [(poly1[i] + poly2[i]) % p for i in range(max_len)]

def multiply_polynomials(poly1, poly2, p):
    result = [0] * (len(poly1) + len(poly2) - 1)
    for i in range(len(poly1)):
        for j in range(len(poly2)):
            result[i + j] = (result[i + j] + poly1[i] * poly2[j]) % p
    return result

def find_generator(p):
    for g in range(2, p):
        seen = {pow(g, i, p) for i in range(p - 1)}
        if len(seen) == p - 1:
            return g
    raise ValueError("No generator found")

def polynomial_to_string(coefficients):
    terms = [
        f"{coef}" if i == 0 else f"{coef}x^{i}"
        for i, coef in enumerate(coefficients) if coef != 0
    ]
    return " + ".join(terms)

def evaluate_polynomial(coefficients, x, p):
    result = 0
    for i, coef in enumerate(coefficients):
        result = (result + coef * pow(x, i, p)) % p
    return result

## RSCODE base on lagrange interpolate

In [3]:
def lagrange_basis(points, j, p):
    """
    Computes the Lagrange basis function L_j(x).
    :param points: List of interpolation points (x, y)
    :param j: Index of the current basis function
    :param p: Prime number defining the finite field
    :return: Coefficients of L_j(x) (from lowest to highest degree)
    """
    x_j = points[j][0]
    numerator = [1]  # Initialize numerator
    denominator = 1  # Initialize denominator

    for i, (x_i, _) in enumerate(points):
        if i != j:
            numerator = multiply_polynomials(numerator, [-x_i % p, 1], p)
            denominator = (denominator * (x_j - x_i)) % p

    denominator_inv = pow(denominator, -1, p)  # Modular inverse of denominator
    return [(coeff * denominator_inv) % p for coeff in numerator]

def interpolate_polynomial(points, p):
    result = [0] * len(points)
    for j, (_, y_j) in enumerate(points):
        basis = lagrange_basis(points, j, p)
        scaled_basis = [(coeff * y_j) % p for coeff in basis]
        result = add_polynomials(result, scaled_basis, p)
    return result

def rs_encode(message, n, p):
    """
    Reed-Solomon encoding.
    :param n: Codeword length
    :return: Codeword
    """
    k = len(message)
    if n < k:
        raise ValueError("Codeword length n must be greater than or equal to message length k")
    
    g = find_generator(p)
    points = [(pow(g, i, p), message[i]) for i in range(k)]
    coefficients = interpolate_polynomial(points, p)

    for x, y in points:
        assert evaluate_polynomial(coefficients, x, p) == y, f"Interpolation error: x={x}, y={y}"

    return [evaluate_polynomial(coefficients, pow(g, i, p), p) for i in range(n)]

def rs_decode(codeword, m):
    """
    In our case the rscode issystematic,
    meaning for any message m, the first m symbols of recode(m) are the entries of m itself.
    So we return the first m elements in codeword
    """
    return codeword[:m]

In [4]:
# Simple test
codeword = rs_encode(test_message, code_word_length, prime)

print(f"Raw messages: {test_message}")
print(f"Reed-Solomon Codeword: {codeword}")

assert test_message == rs_decode(codeword, message_length)

Raw messages: [3, 7, 11]
Reed-Solomon Codeword: [3, 7, 11, 7, 45, 40]


# Brakedown

The key point of Brakedown is identifying Tensor Product Structure in Polynomial Evaluation Queries

Explanation

TODO

## Public function

In [5]:
def tensor_form(z, deg, p):
    """
    Generates the tensor form of z based on the polynomial degree.
    :return: Vectors a and b representing the tensor form
    """
    m = int((deg + 1)**0.5)
    a = [pow(z, i, p) for i in range(m)]  # a = (1, z, z^2, ..., z^(m-1))
    b = [pow(z, m * i, p) for i in range(m)]  # b = (1, z^m, z^(2m), ..., z^(m(m-1)))
    return a, b

## Prover compoments

In [13]:
def coefficient_to_rscode_matrix(coefficients, m, n, p):
    """
    1. Converts coefficients to matrix form.
    2. Encodes the matrix using Reed-Solomon codes.
    3. Prover claims the result (honest).
    :param coefficients: Polynomial coefficients (low to high degree)
    :param m: Sqrt of (deg(poly) + 1)
    :param n: encoded codeword length
    :param p: Prime number defining the finite field Fp
    :return: Matrix M
    """
    degree = len(coefficients)
    if degree % m != 0:
        raise ValueError(f"Length of coefficients {degree} must be divisible by number of rows {m}")

    row_size = degree // m  # Size of each sub-polynomial

    # Split coefficients into m sub-polynomials
    sub_polynomials = [
        coefficients[i * row_size:(i + 1) * row_size]
        for i in range(m)
    ]

    # Encode each sub-polynomial and construct matrix M
    matrix = []
    for sub_poly in sub_polynomials:
        codeword = rs_encode(sub_poly, n, p)
        matrix.append(codeword)

    return np.array(matrix)

def compute_w(b, M, p):
    """
    Computes w = b^T * M, where b is a row vector and M is a matrix.
    :param b: Row vector (can be part of the tensor form or a randomly generated vector by verifier)
    :param M: Input matrix
    :param p: Prime number defining the finite field
    :return: Resultant vector w after modular arithmetic
    """
    return [(sum(b[i] * M[i][j] for i in range(len(b))) % p) for j in range(M.shape[1])]

def decode_w(w):
    """
    Decodes the vector w.
    :param w: Vector w
    :return: Decoded message
    """
    return rs_decode(w, message_length)

## Verifer compoments

In [14]:
def generate_random_vector(m, p):
    """
    Generates a random vector r over the finite field Fp.
    :param m: Sqrt of (deg(poly) + 1)
    :return: Random vector r
    """
    return [random.randint(0, p - 1) for _ in range(m)]

def consistency_check(M, r, w, p, t):
    """
    Verifier performs a consistency check on the columns of the matrix M.
    :param M: Matrix M submitted by the Prover
    :param r: Random vector r
    :param w: Vector w submitted by the Prover
    :param p: Prime number defining the finite field
    :param t: Number of random columns to check
    :return: True if consistent, otherwise False
    """
    n = M.shape[1]  # number of columns of matrix

    # Step 1: random pick columns
    sampled_indices = random.sample(range(n), t)
    print(f"Randomly selected column indices: {sampled_indices}")

    # Step 2: consistency to each selected column
    for i in sampled_indices:
        # calculate r^T * M_i by verifier own
        computed_wi = sum(r[j] * M[j][i] for j in range(len(r))) % p

        # check computed_wi is equal to Prover's w[i]
        if computed_wi != w[i]:
            print(f"Column {i} inconsistency: w[{i}] = {w[i]}, computed = {computed_wi}")
            return False

    return True

In [15]:
def simulate_interaction(coefficients, z, m, n, p, t):
    """
    Simulates the complete interaction between Prover and Verifier
    :param m: Sqrt of (deg(poly) + 1)
    :param n: encoded codeword length, also, number of columns in matrix M
    :param t: Number of random columns to check
    :return: the evaluation of p(z)
    """
    ## Commitment Phase
    # Step 1: Prover declares matrix M and evaluates the polynomial at point z
    M = coefficient_to_rscode_matrix(coefficients, m, n, p)
    # Badcase
    # Then verifier will recive MBad to do the consistency_check
    # MBad= coefficient_to_rscode_matrix([4, 7, 11, 5, 2, 8, 4, 9, 1], m, n, p)
    print(f"Prover declares matrix M:\n {M}")

    # Step 2: Verifier generates a random vector r
    r = generate_random_vector(m, p)
    print(f"Verifier generates random vector r: {r}")

    # Step 3: Prover computes w = r^T * M, decodes it, and returns message v
    v = decode_w(compute_w(r, M, p))
    print(f"Prover send message : {v} to verifier")

    # Step 5: Verifier performs consistency check
    w = rs_encode(v, n, p)
    is_consistent = consistency_check(M, r, w, p, t)
    print(f"Consistency check result: {is_consistent}")

    ## Evaluation Phase
    # Step 1: Calculate tensor form of point z which is public to all
    a, b = tensor_form(z, len(coefficients), p)
    print(a, b)

    # Step 2: Prover calculate the left part of tensor product b*U*a which is b*U
    v = decode_w(compute_w(b, M, p))  # Computes v' = b^T * M
    print(f"Prover send message : {v} to verifier")

    # Step 3: Verifier ckecks v_i == b * M_i by random select
    w = rs_encode(v, n, p)
    is_consistent = consistency_check(M, b, w, p, t)
    print("Consistency check result:", is_consistent)

    # Step 4: Verifier calculate the rest part of of tensor product b*U*a which is (b*U)*a
    evaluation = sum(v[i] * a[i] for i in range(len(a))) % p

    return evaluation

In [16]:
# prime = 101
# coefficients = generate_random_polynomial(prime, 8)
result = simulate_interaction(coefficients, open_point, message_length, code_word_length , prime, sec_num)
print("Final result:", result)
assert result == evaluate_polynomial(coefficients, open_point, prime)

Prover declares matrix M:
 [[ 3  7 11  7 45 40]
 [ 5  2  8 46 13  7]
 [ 4  9  1 29 36 36]]
Verifier generates random vector r: [39, 9, 26]
Prover send message : [31, 8, 10] to verifier
Randomly selected column indices: [4, 5]
Consistency check result: True
[1, 5, 25] [1, 31, 21]
Prover send message : [7, 23, 45] to verifier
Randomly selected column indices: [3, 2]
Consistency check result: True
Final result: 25
